# Image Matching and Homography Estimation with OpenCV and LightGlue

In [1]:
import os
import cv2 
import json
import torch
import numpy as np
import matplotlib.pyplot as plt

from lightglue import viz2d
from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
import CSRansac

In [2]:
class cfg:
    img0 = "img0.png"
    img1 = "img1.png"
    
    size = (320, 240)
    interpolation = cv2.INTER_AREA
    
    lightglue = {
        "extractor": "SuperPoint", # SuperPoint, DISK
        "device": "cuda", # cpu, cuda
        "max_kpts": 2048,
        "homography": {
            "method": cv2.RANSAC,
            "ransacReprojThreshold": 3.0
        }
    }

## LightGlue

In [3]:
def load_img(file, size, interpolation):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size, interpolation=interpolation)
    return img


def get_homography(src_pts, dst_pts, method, ransacReprojThreshold):
    homography, mask = cv2.findHomography(
        src_pts, 
        dst_pts, 
        method=method, 
        ransacReprojThreshold=ransacReprojThreshold
    )
    return homography, mask

def match_lightglue(img0, img1, cfg):
    img0 = load_image(img0)
    img1 = load_image(img1)
    
    if cfg["extractor"] == "SuperPoint":
        extractor = SuperPoint(max_num_keypoints=cfg["max_kpts"]).eval().to(cfg["device"])
        # matcher = LightGlue(features='superpoint').eval().to(cfg["device"])
        matcher = LightGlue(features='superpoint', depth_confidence=-1, width_confidence=-1)

    # if cfg["extractor"] == "DISK":
    #     extractor = DISK(max_num_keypoints=cfg["max_kpts"]).eval().to(cfg["device"])  # load the extractor
    #     matcher = LightGlue(features='disk').eval().to(cfg["device"])  # load the matcher

    # extract local features
    feats0 = extractor.extract(img0.to(cfg["device"]))  # auto-resize the image, disable with resize=None
    feats1 = extractor.extract(img1.to(cfg["device"]))
    
    # match the features
    matches01 = matcher({'image0': feats0, 'image1': feats1})
    feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]  # remove batch dimension
    
    # get results
    kpts0 = feats0["keypoints"]
    kpts1 = feats1["keypoints"]
    matches = matches01['matches']  # indices with shape (K,2)
    points0 = kpts0[matches[..., 0]]  # coordinates in img0, shape (K,2)
    points1 = kpts1[matches[..., 1]]  # coordinates in img1, shape (K,2)
        
    return {
        "points0": points0,
        "points1": points1,
        "matches01": matches01, 
        "matches": matches,
        "kpts0": kpts0,
        "kpts1": kpts1,
        "img0": img0,
        "img1": img1
    }

def lightglue2opencv(points0, points1, matches, kpts0, kpts1, img0, img1, **kwargs):
    return {
        "src_pts": points0.cpu().numpy().reshape(-1, 1, 2),
        "dst_pts": points1.cpu().numpy().reshape(-1, 1, 2),
        "kp0": cv2.KeyPoint_convert(kpts0.cpu().numpy()),
        "kp1": cv2.KeyPoint_convert(kpts1.cpu().numpy()),
        "matches": tuple(
            cv2.DMatch(matches[i][0].item(), matches[i][1].item(), 0.) 
            for i in range(matches.shape[0])
        ),
        "img0": cv2.cvtColor((255 * img0).cpu().numpy().astype(np.uint8).transpose(1, 2, 0), cv2.COLOR_RGB2GRAY),
        "img1": cv2.cvtColor((255 * img1).cpu().numpy().astype(np.uint8).transpose(1, 2, 0), cv2.COLOR_RGB2GRAY)
    }

def change_coord(homography_lightglue, x, y):
    H = homography_lightglue
    source_coord = np.array([x, y, 1], dtype='float32')
    transformed_coord = np.dot(H, source_coord.T)
    transformed_coord = transformed_coord / transformed_coord[2] # 정규 좌표로 변환
    transformed_coord = transformed_coord.astype('float32')
    transformed_coord_2d = transformed_coord[:2].tolist()
    
    return transformed_coord_2d

In [5]:
import random
import math
    
def csransac(target_keypoint, frame_keypoint):

    # input: np.array([[t_x1,t_y1],[t_x2,t_y2],...]), np.array([[f_x1,f_y1],[f_x2,f_y2],...])
    # [t_x1,t_y1] 과 [f_x1,f_y1]은 매칭된 특징점.
    # 매칭된 특징점의 인덱스가 같아야함.

    homography = [[1,0,0],[0,1,0],[0,0,1]]
    best_homography = [[1,0,0],[0,1,0],[0,0,1]]
    if len(target_keypoint) < 6: # 매칭된 특징점의 수가 5개 이하일 경우 호모그래피를 추정할 수 없다고 판단.
        return np.array([[1,0,0],[0,1,0],[0,0,1]]), 0
    
    max_iteration = 1000
    iteration = 1000
    iteration_count = 0
    max_g = 1000
    p = 0.99
    s = 4
    best_inlier_rate = -1
    
    while 1:
        if iteration_count >= iteration:
            break
        elif iteration_count >= max_iteration:
            break
        
        satisfaction = False
        count = 0
        
        while satisfaction == False:
            target_sample = list()
            frame_sample = list()
            grid_list = list()
            
            for i in range(34):
                grid_list.append(-1)
                
            for i in range(4):
                count += 1
                
                if count > max_g:
                    for top_idx in range(4):
                        target_sample.append(target_keypoint[top_idx])
                        frame_sample.append(frame_keypoint[top_idx])
                    break
                
                idx = random.randint(0, len(target_keypoint)-1)
                target_sample.append(target_keypoint[idx])
                frame_sample.append(frame_keypoint[idx])
                col, row = get_position(frame_keypoint[idx])
                #print(col, row)
                grid_list[row] = col
            #==================================================
            #CSP
            m_i = 0
            m_j = 0
            for i in range(34):
                m_i = i
                
                if (grid_list[i] == -1):
                    continue
                
                for j in range(34):
                    m_j = j
                    
                    if i == j:
                        continue
                    if (grid_list[j] == -1):
                        continue
                    if grid_list[i] == grid_list[j]:
                        break
                    d = i - j
                    if (grid_list[i] == (grid_list[j] - d)) or (grid_list[i] == (grid_list[j] + d)):
                        break
                    if d == 1 or d == -1:
                        if grid_list[i] == grid_list[j] + 2 or grid_list[i] == grid_list[j] - 2:
                            break
                    if d == 2 or d == -2:
                        if grid_list[i] == grid_list[j] + 1 or grid_list[i] == grid_list[j] - 1:
                            break
                if m_j != 33:
                    break
            if m_i == 33:
                satisfaction = True
            #==================================================
        
        homography = find_homography(target_sample, frame_sample)
        inliers = calculate_inliers(homography, target_keypoint, frame_keypoint,5)
        inlier_rate = inliers / len(target_keypoint)
        if inlier_rate == 1:
            break
        if inlier_rate > best_inlier_rate:
            best_inlier_rate = inlier_rate
            best_homography = homography
        e = 1 - best_inlier_rate
        try:
            iteration = math.log(1.0 - p) / math.log(1.0 - math.pow(1.0 - e, s))
        except:
            iteration = iteration
        iteration_count += 1
    if best_inlier_rate <= 0.3: # best_inlier_rate <= 0.3 일 경우 호모그래피 추정에 실패했다고 판단.
        return np.array([[1,0,0],[0,1,0],[0,0,1]]), best_inlier_rate
    return best_homography, best_inlier_rate
	
def get_position(keypoint):
    col_size = 34 # csp ransac grid size
    row_size = 34 # csp ransac grid size
    col = int(keypoint[0] / (1280 / col_size)) # 원본 이미지 width
    row = int(keypoint[1] / (960 / row_size)) # 원본 이미지 height
    if col == col_size:
        col -= 1
    if row == row_size:
        row -= 1
    return (col, row)
    return [item for row in matrix for item in row]

def perspective_transform(src, homo):
    w = homo[2, 0] * src[0] + homo[2, 1] * src[1] + homo[2, 2]
    x = (homo[0, 0] * src[0] + homo[0, 1] * src[1] + homo[0, 2]) / w
    y = (homo[1, 0] * src[0] + homo[1, 1] * src[1] + homo[1, 2]) / w
    return (x, y)

def flatten_comprehension(matrix):
    return [item for row in matrix for item in row]

def find_homography(src_points, dst_points):
    A = []
    for i in range(4):
        X, Y = src_points[i][0], src_points[i][1]
        x, y = dst_points[i][0], dst_points[i][1]
        A.append([-X, -Y, -1, 0, 0, 0, x*X, x*Y, x])
        A.append([0, 0, 0, -X, -Y, -1, y*X, y*Y, y])

    A = np.asarray(A)
    U, S, Vh = np.linalg.svd(A)
    L = Vh[-1, :] / Vh[-1, -1]
    H = L.reshape(3, 3)
    return H

def calculate_inliers(H, points1, points2, threshold):
    num_points = len(points1)
    points1_hom = np.concatenate([points1, np.ones((num_points, 1))], axis=1).T
    points2 = np.array(points2)
    estimates = np.dot(H, points1_hom)
    estimates /= estimates[2, :]
    errors = np.sqrt(np.sum((points2.T - estimates[:2, :]) ** 2, axis=0))
    inliers = np.sum(errors <= threshold)
    return inliers
    
def estimate_scale_change(old_points, new_points):
    old_center = np.mean(old_points, axis=0)
    new_center = np.mean(new_points, axis=0)
    old_dists = np.linalg.norm(old_points - old_center, axis=1)
    new_dists = np.linalg.norm(new_points - new_center, axis=1)
    return np.mean(new_dists) / np.mean(old_dists)

## Dataset 전처리

In [2]:
aircraft_datasets = "D:/aircraft_datasets"

lables = os.listdir(aircraft_datasets + "/label")
videos = os.listdir(aircraft_datasets + "/video")

In [4]:
origin_coordinate = []

# 원점 좌표값 불러오기
for label in lables:
    with open(aircraft_datasets + "/label/" + label, "r") as f:
        json_file = json.load(f)
        coord = json_file["targetAnnotation"]
        coord[0] = coord[0] * 640
        coord[1] = coord[1] * 480
        origin_coordinate.append(coord)
        
print(origin_coordinate)
print(len(origin_coordinate))

[[319.171968, 270.55248], [320.0, 265.24536], [344.464896, 256.02912], [313.576128, 257.29579199999995], [325.48172800000003, 168.083808], [315.939648, 202.48910399999997], [325.479232, 168.080352], [312.391232, 306.426768], [320.0, 265.23864], [331.487168, 26.902847999999988], [316.5232, 203.087808], [329.47750399999995, 59.02296000000001], [320.0, 337.57583999999997], [324.136448, 161.35992000000002], [309.34656, 253.744368], [321.263104, 248.872656], [332.852352, 236.02262399999998], [326.04812799999996, 203.801712], [318.48947200000003, 251.060496], [320.964672, 255.825552], [321.25523200000003, 215.70609599999997], [319.453312, 225.751632], [319.45344, 180.868992], [321.200512, 215.63779200000002], [321.227712, 215.671728], [316.37516800000003, 230.084016], [316.20556799999997, 231.432768], [320.89824, 312.286224], [320.950912, 198.62135999999998], [315.928128, 231.49977600000003], [320.895168, 257.614128], [320.82163199999997, 257.47713600000003], [320.820608, 257.477952], [320.6

In [6]:
video_dir = os.path.join(aircraft_datasets, "video")
output_dir = os.path.join(aircraft_datasets, "frames_from_video")

In [12]:
#동영상에서 각 프레임을 이미지 파일로 저장하는 코드

# 동영상 파일 로드
video = cv2.VideoCapture('stable_demo_video.mp4')

# 프레임 카운터 초기화
frame_count = 0

while True:
    # 동영상에서 프레임을 읽음
    ret, frame = video.read()
    if not ret:
        break  # 동영상 끝에 도달하면 중단
    
    # 프레임을 이미지 파일로 저장
    frame_filename = os.path.join('stable_video', f'frame_{frame_count:04d}.jpg')
    frame = cv2.resize(frame, (640, 480))
    cv2.imwrite(frame_filename, frame)
    
    frame_count += 1

# 자원 해제
video.release()

In [12]:
# 모든 폴더의 이미지 수를 저장할 리스트
images = []

# output_dir 내의 모든 폴더에 대한 반복
for folder_name in os.listdir(output_dir):
    folder_path = os.path.join(output_dir, folder_name)
    
    # 폴더 내 파일 및 하위 폴더의 갯수를 세어 이미지 리스트에 추가
    num_files = len(os.listdir(folder_path))
    images.append(num_files)

# images 리스트의 길이 반환
num_images = len(images)
print(f"총 이미지 수: {num_images}")

[['D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0000.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0001.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0002.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0003.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0004.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0005.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0006.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0007.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0008.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0009.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0010.jpg', 'D:/aircraft_datasets\\frames_from_video\\FuelPumpRemoval_00001\\frame_0011.jpg', 'D:/aircraft_d

In [11]:
from vidstab import VidStab

# Using defaults
stabilizer = VidStab()
stabilizer.stabilize(input_path='demo_video.mp4', output_path='stable_demo_video.mp4')

# # Using a specific keypoint detector
# stabilizer = VidStab(kp_method='ORB')
# stabilizer.stabilize(input_path='input_video.mp4', output_path='stable_video.avi')

# # Using a specific keypoint detector and customizing keypoint parameters
# stabilizer = VidStab(kp_method='FAST', threshold=42, nonmaxSuppression=False)
# stabilizer.stabilize(input_path='input_video.mov', output_path='stable_video.avi')

FuelPumpRemoval_00001.mp4 stabilized and saved to D:/aircraft_datasets\stabilized_frame\FuelPumpRemoval_00001.mp4
FuelPumpRemoval_00002.mp4 stabilized and saved to D:/aircraft_datasets\stabilized_frame\FuelPumpRemoval_00002.mp4
FuelPumpRemoval_00003.mp4 stabilized and saved to D:/aircraft_datasets\stabilized_frame\FuelPumpRemoval_00003.mp4
FuelPumpRemoval_00004.mp4 stabilized and saved to D:/aircraft_datasets\stabilized_frame\FuelPumpRemoval_00004.mp4
FuelPumpRemoval_00005.mp4 stabilized and saved to D:/aircraft_datasets\stabilized_frame\FuelPumpRemoval_00005.mp4
FuelPumpRemoval_00006.mp4 stabilized and saved to D:/aircraft_datasets\stabilized_frame\FuelPumpRemoval_00006.mp4
FuelPumpRemoval_00007.mp4 stabilized and saved to D:/aircraft_datasets\stabilized_frame\FuelPumpRemoval_00007.mp4
FuelPumpRemoval_00008.mp4 stabilized and saved to D:/aircraft_datasets\stabilized_frame\FuelPumpRemoval_00008.mp4
FuelPumpRemoval_00009.mp4 stabilized and saved to D:/aircraft_datasets\stabilized_frame\

In [ ]:
images

## Compare Homography Matrices

In [9]:
#원본 이미지를 기준으로 좌표 변환
len_img = len(images)

coord_list = []
img0 = images[0]

for i in range(len_img):
    if i != len_img - 1:
        img1 = images[i+1]
        
        # LightGlue
        results_lightglue = match_lightglue(img0, img1, cfg.lightglue)
        target_keypoint = results_lightglue["points0"].cpu().numpy()
        frame_keypoint = results_lightglue["points1"].cpu().numpy()
        
        homography, _ = csransac(target_keypoint, frame_keypoint)
        projected_pts = perspective_transform(np.array([x, y]), homography)
        
        coord_list.append(projected_pts)

In [60]:
# #인접한 프레임들끼리의 좌표 변환
# len_img = len(images)
# x = 637 // 2
# y = 367 // 2

# coord_list = []

# for i in range(len_img):
#     if i != len_img - 1:
#         img0 = images[i]
#         img1 = images[i+1]
        
#         # LightGlue
#         results_lightglue = match_lightglue(img0, img1, cfg.lightglue)
#         results_lightglue_opencv = lightglue2opencv(**results_lightglue)
#         src_pts = results_lightglue_opencv["src_pts"]
#         dst_pts = results_lightglue_opencv["dst_pts"]
#         # target_keypoint = results_lightglue["points0"].cpu().numpy()
#         # frame_keypoint = results_lightglue["points1"].cpu().numpy()
        
#         homography, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 3.0)
#         # homography, _ = csransac(target_keypoint, frame_keypoint)
#         _coord = perspective_transform(np.array([x, y]), homography)
#         # x = _coord[0][0][0]
#         # y = _coord[0][0][1]
#         #_coord = change_coord(homography, x, y)
#         x = _coord[0]
#         y = _coord[1]
        
#         coord_list.append(_coord)

In [10]:
# 폴더 경로 설정
folder = 'video'

# 동영상 저장 경로 설정
output_video_path = 'result_origin.mp4'

# 동영상 속성 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 설정 (XVID를 사용하면 AVI 형식으로 저장)
fps = 30.0  # 초당 프레임 수
frame_width = 640  # 프레임 너비
frame_height = 480  # 프레임 높이

out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

x = 637 // 2
y = 367 // 2

i = 0
for name in os.listdir(folder):
    img = cv2.imread(os.path.join(folder, name))
    if i == 0:
        cv2.circle(img, (x, y), 3, (0, 0, 255), -1)
    elif i == len_img - 1:
        break
    else:
        x = round(coord_list[i][0])
        y = round(coord_list[i][1])
        cv2.circle(img, (x, y), 3, (0, 0, 255), -1)

    i = i + 1
    
    # 프레임을 동영상에 추가
    out.write(img)

# 동영상 저장 종료
out.release()


318 183


## Error Estimate

In [20]:
coord_x = []
coord_y = []

for i in range(len(coord_list)):
    x = coord_list[i][0]
    y = coord_list[i][1]
    
    x = x / 640
    y = y / 480
    
    x = round(x, 3)
    y = round(y, 3)
    
    coord_x.append(x)
    coord_y.append(y)
    
print(coord_x)

[0.496, 0.495, 0.497, 0.491, 0.495, 0.495, 0.493, 0.494, 0.491, 0.494, 0.492, 0.491, 0.49, 0.49, 0.487, 0.483, 0.487, 0.483, 0.482, 0.488, 0.485, 0.473, 0.484, 0.478, 0.48, 0.49, 0.474, 0.475, 0.475, 0.484, 0.481, 0.48, 0.485, 0.474, 0.48, 0.478, 0.476, 0.471, 0.463, 0.476, 0.477, 0.466, 0.472, 0.465, 0.457, 0.458, 0.471, 0.463, 0.457, 0.453, 0.467, 0.467, 0.454, 0.463, 0.455, 0.461, 0.465, 0.463, 0.461, 0.462, 0.462, 0.465, 0.497, 0.453, 0.454, 0.455, 0.446, 0.456, 0.497, 0.497, 0.46, 0.449, 0.454, 0.497, 0.46, 0.497, 0.497, 0.497, 0.497, 0.497, 0.454, 0.497, 0.456, 0.456, 0.452, 0.446, 0.441, 0.447, 0.443, 0.443, 0.455, 0.444, 0.453, 0.445, 0.444, 0.441, 0.497, 0.448, 0.456, 0.454, 0.497, 0.497, 0.451, 0.456, 0.497, 0.459, 0.497, 0.497, 0.497, 0.462, 0.452, 0.464, 0.441, 0.461, 0.455, 0.445, 0.452, 0.462, 0.455, 0.461, 0.497, 0.453, 0.464, 0.464, 0.453, 0.459, 0.451, 0.459, 0.468, 0.456, 0.458, 0.45, 0.458, 0.461, 0.46, 0.469, 0.459, 0.461, 0.462, 0.464, 0.467, 0.476, 0.467, 0.476, 0

In [21]:
x = 637 // 2
y = 367 // 2

x = x / 640
y = y / 480

x = round(x, 3)
y = round(y, 3)

print(x, y)

0.497 0.381


In [25]:
min_x = x - 0.05
max_x = x + 0.05

min_y = y - 0.05
max_y = y + 0.05

num_error = 0
pixel_error = 0
_pixel_error = 0

for i in range(len(coord_x)):
    if coord_x[i] < min_x or coord_x[i] > max_x:
        num_error += 1
    elif coord_y[i] < min_y or coord_y[i] > max_y:
        num_error += 1
    
    #원점 x, y로부터의 픽셀 거리
    _pixel_error += math.sqrt((coord_x[i] - x)**2 + (coord_y[i] - y)**2)    
    if _pixel_error > pixel_error:
        pixel_error = _pixel_error
    
    
    
print(num_error)
print(pixel_error)

11
7.493632979142081
